In [1]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [6]:
import joblib
import mtcnn
from PIL import Image
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print(tf.__version__)

from tensorflow.keras.models import load_model
# model = load_model('D:/Down/facenet/model/facenet_keras.h5')


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5534188251917773160
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7129736807
locality {
  bus_id: 1
  links {
  }
}
incarnation: 5300854626103977939
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
2.1.0


In [7]:
def extract_face(filename, required_size=(114, 114)):
    image = Image.open(filename)
    image = image.convert('RGB')
    pixels = np.asarray(image)
    detector = MTCNN()
    results = detector.detect_faces(pixels)
    if results!=[]:
        x1, y1, width, height = results[0]['box']
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        face = pixels[y1:y2, x1:x2]
        image = Image.fromarray(face)
        image = image.resize(required_size)
        face_array = np.asarray(image)
        return face_array

def get_embedding(model, face_pixels):
    face_pixels = face_pixels.astype('int32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    samples = np.expand_dims(face_pixels, axis=0)
    yhat = model.predict(samples)
    return yhat[0]


In [3]:
# model = load_model('D:/Down/facenet/model/facenet_keras.h5')

In [8]:
path = "D:/Down/inha_data (2)/inha_data"
os.chdir(path)

In [9]:
all_path = []
def trainpath(address):
    for i in os.listdir(address):
        if os.path.isdir(address + "/" + i):
            trainpath(address + "/" + i)
        else:
            all_path.append(address +"/" + i)

def dircount(address):
    pathcount = []
    for i in os.listdir(address):
        pathcount.append(len(os.listdir(address + "/" + i)))
    
    return pathcount

In [10]:
trainpath("train")
pathcount = dircount("train")
pathNumber = []
for i in range(len(pathcount)):
    pathNumber.extend([i] * pathcount[i])

In [12]:
over40 = np.array([i for i in range(len(pathcount)) if pathcount[i]>=40 ])
idx_over40 = np.isin(np.array(pathNumber), over40)
idD = np.c_[pathNumber, all_path]
idD = pd.DataFrame(idD)
idD.columns = ["cl", "cl_path"]
idD = idD.loc[idx_over40,:]
idx_group = idD.groupby("cl").groups
unique_cl = list(idx_group.keys())

In [20]:
# make 1000 photos..............
#  1 :10k, 0 : 10k
import random

def randomselect(number, repeat):
    cl1k = random.sample(unique_cl, number+(number*repeat))
    return [cl1k[:number], cl1k[number:]]

# 일단 오른쪽만 가능
def shiftright(x, i):
    x_copy = x[:]
    for k in range(i, len(x)):
        x_copy[k] = x[k-i]
    for k in range(i):
        x_copy[k] = x[-i+k]
    return x_copy

def samedat(r_number, data):
    cl1k_same = data
    same_left = []; same_right=[]
    for j in cl1k_same:
        same_idx = idx_group[j]
        temp_left = random.sample(same_idx.to_list(), r_number)
        same_left.extend(temp_left)
        same_right.extend(shiftright(temp_left, 2))
        
    return [same_left, same_right]

def differdat(data):
    cl1k_differ = data
    differ_left = []; differ_right = []
    for j in cl1k_differ:
        differ_idx = idx_group[j]
        temp_left = random.sample(differ_idx.to_list(), 1)
        differ_left.extend(temp_left)
    differ_right.extend(shiftright(differ_left, 2))
        
    return [differ_left, differ_right]

In [21]:
data = randomselect(50,10)
idx_same = samedat(10, data[0])
idx_differ = differdat(data[1])

In [22]:
def runloop(data):
    res = []
    for i in data:
        res.append(extract_face(idD.cl_path[i]))
    return res

In [133]:
with tf.device("/GPU:0"):
    same_left = runloop(idx_same[0])
    same_right = runloop(idx_same[1])

    differ_left =runloop(idx_differ[0])
    differ_right = runloop(idx_differ[1])

In [135]:
len(removeNULL(same_left))

498

In [130]:
def removeNULL(list1):
    return [i for i in list1 if type(i)!=type(None)]

In [142]:
r_same_left = np.array(removeNULL(same_left))
r_same_right = np.array(removeNULL(same_right))
r_differ_left = np.array(removeNULL(differ_left))
r_differ_right = np.array(removeNULL(differ_right))

In [197]:
em_same_left = [get_embedding(model, r_same_left[i]) for i in range(len(r_same_left))]
em_same_right = [get_embedding(model, r_same_left[i]) for i in range(len(r_same_right))]
em_differ_left = [get_embedding(model, r_same_left[i]) for i in range(len(r_differ_left))]
em_differ_right = [get_embedding(model, r_same_left[i]) for i in range(len(r_differ_right))]


In [222]:
trainX = pd.DataFrame(np.r_[np.c_[em_same_left, em_same_right], np.c_[em_differ_left, em_differ_right]])
trainy = np.array([1] * len(em_same_left) + [0] * len(em_differ_left))

In [242]:
from sklearn.ensemble import RandomForestClassifier
import sklearn
forest = RandomForestClassifier(n_estimators=11, max_depth=20)
forest.fit(trainX,trainy)

RandomForestClassifier(max_depth=20, n_estimators=11)

In [338]:
test_left = []
test_right = []
for i in os.listdir("test"):
    p1 = re.compile("left")
    if p1.findall(i) == ['left']:
        p2 = re.compile("[0-9]")
        test_left.append(int("".join(p2.findall(i))))
    else:
        p3 = re.compile("[0-9]")
        test_right.append(int("".join(p3.findall(i))))        

test_left.sort()
test_right.sort()
test_left = list(map( lambda x :"test/left_face_"+str(x)+".jpg", test_left))
test_right = list(map( lambda x :"test/right_face_"+ str(x)+".jpg", test_right))

In [ ]:
# remove None!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [379]:
rcg_test_left = []
rcg_test_right = []

for i in range(len(test_left)):
    rcg_test_left.append(extract_face(test_left[i]))
    rcg_test_right.append(extract_face(test_right[i]))
    print(i)

0


KeyboardInterrupt: 

5